In [1]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [3]:
root_dir = "/gdrive/My Drive/CV/c1"

import sys
sys.path.append(root_dir)

In [4]:
# 훈련

import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from skimage.feature import hog
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

# 이미지 경로에서 데이터를 로드합니다.
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img = cv2.imread(os.path.join(label_folder, filename))
            if img is not None:
                images.append(img)
                labels.append(label)
    return images, labels

# 이미지 전처리를 수행합니다.
def preprocess_image(image):
    # Histogram Equalization
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    # Gaussian Blur
    img_output = cv2.GaussianBlur(img_output, (5, 5), 0)
    
    return img_output


def extract_features(image):
    image_resized = cv2.resize(image, (128, 128))

    # Convert the image to gray scale
    gray = rgb2gray(image)
    
    # Gray Scale Histogram
    hist = np.histogram(gray, bins=256, range=(0.0, 1.0))[0]
    
    # GLCM
    gray = img_as_ubyte(gray)
    glcm = greycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(glcm, prop='contrast')[0, 0]
    
    # HOG
    fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
    
    # Ensuring the HOG descriptor size is fixed
    fd = fd[:256]
    
    # Concatenate features into a single numpy array
    feature = np.concatenate([hist, np.array([contrast]), fd], axis=0)
    
    return feature




In [7]:
 #데이터셋을 로드합니다.
images, labels = load_images_from_folder(root_dir + '/dataset')

# 레이블을 숫자로 변환합니다.
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# 전처리 및 특징 추출을 수행합니다.
processed_images = [preprocess_image(img) for img in images]
features = [extract_features(img) for img in processed_images]

print(len(features))
print(len(labels))

feature_lengths = [len(feat) for feat in features]
print('Feature lengths:', feature_lengths)

# KFold를 설정합니다.
kf = KFold(n_splits=5, shuffle=True, random_state=42)

model_save_path = '/gdrive/My Drive/CV/c1/models/task2/try1'

# 각 split마다 KNN 모델을 학습하고 평가합니다.
# 각 split마다 KNN 모델을 학습하고 평가합니다.
for fold, (train_index, test_index) in enumerate(kf.split(features)):
    X_train = np.array(features)[train_index]
    X_test = np.array(features)[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    model = KNeighborsClassifier(n_neighbors=10)
    model.fit(X_train, y_train)

    model_path = root_dir + 'models/task2/try1'
    
    # Ensure the directory exists
    os.makedirs(model_save_path, exist_ok=True)
    
    # Save the model in the specified directory
    joblib.dump(model, f'{model_save_path}/model_{fold}.pkl')

    y_pred = model.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # Add this line to print the classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Streaming output truncated to the last 5000 lines.
<ipython-input-4-cf393f1f873a>:58: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
<ipython-input-4-cf393f1f873a>:58: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` 

10208
10208
Feature lengths: [513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.41625857002938293
Classification Report:
               precision    recall  f1-score   support

      Bicycle       0.19      0.10      0.13       141
       Bridge       0.31      0.04      0.07       105
          Bus       0.32      0.20      0.24       231
          Car       0.42      0.85      0.56       727
      Chimney       0.00      0.00      0.00        15
    Crosswalk       0.49      0.18      0.26       252
      Hydrant       0.60      0.45      0.51       193
   Motorcycle       0.00      0.00      0.00        23
         Palm       0.37      0.13      0.19       195
Traffic Light       0.28      0.07      0.11       160

     accuracy                           0.42      2042
    macro avg       0.30      0.20      0.21      2042
 weighted avg       0.39      0.42      0.35      2042



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4086232239098481
Classification Report:
               precision    recall  f1-score   support

      Bicycle       0.28      0.13      0.18       160
       Bridge       0.16      0.03      0.05       103
          Bus       0.32      0.19      0.24       259
          Car       0.42      0.85      0.56       713
      Chimney       0.00      0.00      0.00        14
    Crosswalk       0.48      0.19      0.28       238
      Hydrant       0.45      0.46      0.45       162
   Motorcycle       0.00      0.00      0.00        22
         Palm       0.39      0.17      0.23       193
Traffic Light       0.26      0.03      0.06       177

     accuracy                           0.41      2041
    macro avg       0.28      0.20      0.20      2041
 weighted avg       0.37      0.41      0.34      2041



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.404703576678099
Classification Report:
               precision    recall  f1-score   support

      Bicycle       0.12      0.06      0.08       150
       Bridge       0.00      0.00      0.00       105
          Bus       0.38      0.21      0.27       271
          Car       0.42      0.85      0.56       697
      Chimney       0.00      0.00      0.00        12
    Crosswalk       0.40      0.19      0.26       242
      Hydrant       0.53      0.41      0.46       193
   Motorcycle       0.00      0.00      0.00        23
         Palm       0.36      0.17      0.23       182
Traffic Light       0.29      0.05      0.08       166

     accuracy                           0.40      2041
    macro avg       0.25      0.19      0.19      2041
 weighted avg       0.36      0.40      0.33      2041



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# task 2

import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from skimage.feature import hog
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

# 이미지 경로에서 데이터를 로드합니다.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

# 이미지 전처리를 수행합니다.
def preprocess_image(image):
    # Histogram Equalization
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    # Gaussian Blur
    img_output = cv2.GaussianBlur(img_output, (5, 5), 0)
    
    return img_output


def extract_features(image):
    image_resized = cv2.resize(image, (128, 128))

    # Convert the image to gray scale
    gray = rgb2gray(image)
    
    # Gray Scale Histogram
    hist = np.histogram(gray, bins=256, range=(0.0, 1.0))[0]
    
    # GLCM
    gray = img_as_ubyte(gray)
    glcm = greycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(glcm, prop='contrast')[0, 0]
    
    # HOG
    fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
    
    # Ensuring the HOG descriptor size is fixed
    fd = fd[:256]
    
    # Concatenate features into a single numpy array
    feature = np.concatenate([hist, np.array([contrast]), fd], axis=0)
    
    return feature


data_dir = root_dir + '/query'
# 데이터셋을 로드합니다.
test_images = load_images_from_folder(data_dir)

processed_images = [preprocess_image(img) for img in test_images]
test_features = [extract_features(img) for img in processed_images]

model_dir = root_dir + '/models/task2/try1/model_0.pkl'

classifier = joblib.load(model_dir)

predict_labels = []
# labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Palm', 'Traffic Light']
for feature in test_features:
    neigh_ind = classifier.kneighbors(X=test_features, n_neighbors = 10, return_distance=False)
    neigh_labels = np.array(labels)[neigh_ind]

print(neigh_labels)


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
<ipython-input-8-3628335b9f80>:54: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  r

[['Crosswalk' 'Car' 'Car' 'Bus' 'Motorcycle' 'Motorcycle' 'Hydrant'
  'Bicycle' 'Bus' 'Car']
 ['Car' 'Crosswalk' 'Crosswalk' 'Car' 'Hydrant' 'Car' 'Car' 'Hydrant'
  'Car' 'Car']
 ['Hydrant' 'Bus' 'Hydrant' 'Hydrant' 'Car' 'Car' 'Hydrant' 'Bus' 'Bus'
  'Car']
 ['Hydrant' 'Hydrant' 'Bus' 'Hydrant' 'Motorcycle' 'Bus' 'Bicycle'
  'Hydrant' 'Bicycle' 'Bicycle']
 ['Car' 'Motorcycle' 'Car' 'Car' 'Bus' 'Hydrant' 'Crosswalk' 'Bicycle'
  'Hydrant' 'Bicycle']
 ['Car' 'Car' 'Bus' 'Car' 'Bus' 'Car' 'Bridge' 'Motorcycle' 'Car'
  'Hydrant']
 ['Car' 'Car' 'Car' 'Bicycle' 'Car' 'Hydrant' 'Crosswalk' 'Car' 'Bridge'
  'Bus']
 ['Bus' 'Bus' 'Car' 'Car' 'Hydrant' 'Car' 'Car' 'Bridge' 'Crosswalk'
  'Car']
 ['Car' 'Car' 'Car' 'Car' 'Car' 'Car' 'Hydrant' 'Car' 'Hydrant' 'Car']
 ['Car' 'Car' 'Crosswalk' 'Car' 'Bridge' 'Crosswalk' 'Crosswalk' 'Car'
  'Hydrant' 'Bus']
 ['Car' 'Car' 'Car' 'Bicycle' 'Hydrant' 'Car' 'Bicycle' 'Car' 'Car' 'Bus']
 ['Bus' 'Car' 'Crosswalk' 'Bus' 'Hydrant' 'Car' 'Bus' 'Bicycle' 'Bicycle

In [10]:
# task2 

import csv

with open(root_dir + '/c1_t2_a1.csv', 'w')as file:
    write = csv.writer(file)
    for i, neigh_label in enumerate(neigh_labels):
        write.writerow([f'query{i+1}.png'] + list(neigh_label))

In [ ]:
# task 1

import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from skimage.feature import hog
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

# 이미지 경로에서 데이터를 로드합니다.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

# 이미지 전처리를 수행합니다.
def preprocess_image(image):
    # Histogram Equalization
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    # Gaussian Blur
    img_output = cv2.GaussianBlur(img_output, (5, 5), 0)
    
    return img_output


def extract_features(image):
    image_resized = cv2.resize(image, (128, 128))

    # Convert the image to gray scale
    gray = rgb2gray(image)
    
    # Gray Scale Histogram
    hist = np.histogram(gray, bins=256, range=(0.0, 1.0))[0]
    
    # GLCM
    gray = img_as_ubyte(gray)
    glcm = greycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(glcm, prop='contrast')[0, 0]
    
    # HOG
    fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
    
    # Ensuring the HOG descriptor size is fixed
    fd = fd[:256]
    
    # Concatenate features into a single numpy array
    feature = np.concatenate([hist, np.array([contrast]), fd], axis=0)
    
    return feature


data_dir = root_dir + '/query'
# 데이터셋을 로드합니다.
test_images = load_images_from_folder(data_dir)

processed_images = [preprocess_image(img) for img in test_images]
test_features = [extract_features(img) for img in processed_images]

model_dir = root_dir + '/models/try2/model_1.pkl'

classifier = joblib.load(model_dir)

predict_labels = []
labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Palm', 'Traffic Light']
for feature in test_features:
    predict_label = classifier.predict([feature])[0]
    predict_labels.append(labels[predict_label])

print(predict_labels)


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
<ipython-input-5-20ebd0a45d1b>:54: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  r

['Car', 'Bus', 'Hydrant', 'Hydrant', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Crosswalk', 'Bus', 'Car', 'Hydrant', 'Bus', 'Hydrant', 'Crosswalk', 'Car', 'Car', 'Car', 'Bus', 'Bridge', 'Car', 'Bicycle', 'Bicycle', 'Car', 'Car', 'Palm', 'Palm', 'Car', 'Crosswalk', 'Crosswalk', 'Car', 'Hydrant', 'Bridge', 'Bridge', 'Palm', 'Car', 'Car', 'Bridge', 'Car', 'Car', 'Hydrant', 'Car', 'Bridge', 'Car', 'Car', 'Bus', 'Car', 'Bridge', 'Car', 'Palm', 'Car', 'Car', 'Car', 'Bus', 'Hydrant', 'Car', 'Crosswalk', 'Hydrant', 'Car', 'Bridge', 'Bicycle', 'Hydrant', 'Bus', 'Car', 'Car', 'Bicycle', 'Bus', 'Car', 'Hydrant', 'Car', 'Traffic Light', 'Bicycle', 'Car', 'Car', 'Car', 'Palm', 'Bicycle', 'Palm', 'Car', 'Car', 'Car', 'Car', 'Traffic Light', 'Car', 'Car', 'Car', 'Car', 'Palm', 'Car', 'Car', 'Car', 'Bicycle', 'Palm']


In [ ]:
# task 1

import csv

with open(root_dir + '/c1_t1_a1.csv', 'w') as file:
    write = csv.writer(file)
    for i, predict_label in enumerate(predict_labels):
        write.writerow([f'query{i+1}.png', predict_label])